In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

TRAIN_IMG_DIR = '/content/drive/MyDrive/images/train'
TEST_IMG_DIR = '/content/drive/MyDrive/images/test'
TRAIN_CSV_PATH = '/content/drive/MyDrive/train.csv'
TEST_CSV_PATH = '/content/drive/MyDrive/test.csv'
IMAGE_SIZE = 224

In [4]:
train_df = pd.read_csv(TRAIN_CSV_PATH)
test_df = pd.read_csv(TEST_CSV_PATH)

print(f"Train CSV: {train_df.shape}")
print(f"Test CSV: {test_df.shape}")

Train CSV: (16209, 21)
Test CSV: (5404, 20)


In [5]:
def preprocess_tabular(df, is_train=True, scaler=None, encoder=None):
    df_proc = df.copy()

    df_proc = df_proc.drop(['id', 'date', 'lat', 'long'], axis=1, errors='ignore')

    df_proc['age'] = 2025 - df_proc['yr_built']

    df_proc['is_renovated'] = (df_proc['yr_renovated'] > 0).astype(int)
    df_proc['years_since_renovation'] = df_proc.apply(
        lambda x: 2025 - x['yr_renovated'] if x['yr_renovated'] > 0 else 0, axis=1
    )

    df_proc = df_proc.drop(['yr_built', 'yr_renovated'], axis=1)

    target = None
    if is_train and 'price' in df_proc.columns:
        target = df_proc['price'].copy()
        df_proc = df_proc.drop('price', axis=1)

    if 'zipcode' in df_proc.columns:
        if is_train:
            encoder = LabelEncoder()
            df_proc['zipcode'] = encoder.fit_transform(df_proc['zipcode'])
        else:
            df_proc['zipcode'] = encoder.transform(df_proc['zipcode'])

    numeric_cols = df_proc.select_dtypes(include=[np.number]).columns.tolist()

    if is_train or scaler is None:
        scaler = StandardScaler()
        df_proc[numeric_cols] = scaler.fit_transform(df_proc[numeric_cols])
    else:
        df_proc[numeric_cols] = scaler.transform(df_proc[numeric_cols])

    if target is not None:
        df_proc['price'] = target.values

    return df_proc, scaler, encoder

In [6]:
train_tabular, tab_scaler, zip_encoder = preprocess_tabular(train_df, is_train=True)
test_tabular, _, _ = preprocess_tabular(test_df, is_train=False, scaler=tab_scaler, encoder=zip_encoder)

print(f"Processed train tabular: {train_tabular.shape}")
print(f"Processed test tabular: {test_tabular.shape}")

Processed train tabular: (16209, 18)
Processed test tabular: (5404, 17)


In [7]:
def find_image_path(img_id, img_dir):
    img_id_str = str(img_id)
    possible_paths = [
        os.path.join(img_dir, img_id_str),
        os.path.join(img_dir, f"{img_id_str}.jpg"),
        os.path.join(img_dir, f"{img_id_str}.png"),
        os.path.join(img_dir, f"{img_id_str}.jpeg"),
        os.path.join(img_dir, f"{img_id_str}.JPG"),
        os.path.join(img_dir, f"{img_id_str}.PNG"),
        os.path.join(img_dir, f"{img_id_str}.JPEG"),
    ]
    for path in possible_paths:
        if os.path.exists(path):
            return path
    return None

In [8]:
image_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
def load_all_images(img_dir):
    image_dict = {}
    missing_count = 0

    for filename in os.listdir(img_dir):
        file_path = os.path.join(img_dir, filename)

        if os.path.isfile(file_path):
            try:
                img_id = os.path.splitext(filename)[0]
                img = Image.open(file_path).convert('RGB')
                img_tensor = image_transform(img)
                image_dict[img_id] = img_tensor
            except:
                missing_count += 1

    print(f"Loaded {len(image_dict)} images, {missing_count} failed to load")
    return image_dict

In [10]:
train_images = load_all_images(TRAIN_IMG_DIR)


Loaded 16109 images, 1 failed to load


In [1]:
test_images = load_all_images(TEST_IMG_DIR)

NameError: name 'load_all_images' is not defined

In [ ]:
train_tabular_features = train_tabular.drop('price', axis=1).values
train_prices = train_tabular['price'].values
test_tabular_features = test_tabular.values

train_tabular_tensor = torch.tensor(train_tabular_features, dtype=torch.float32)
train_price_tensor = torch.tensor(train_prices, dtype=torch.float32).view(-1, 1)
test_tabular_tensor = torch.tensor(test_tabular_features, dtype=torch.float32)

print(f"Train tabular tensor: {train_tabular_tensor.shape}")
print(f"Train prices tensor: {train_price_tensor.shape}")
print(f"Test tabular tensor: {test_tabular_tensor.shape}")

In [ ]:
train_image_list = list(train_images.values())
test_image_list = list(test_images.values())

train_image_tensor = torch.stack(train_image_list) if train_image_list else None
test_image_tensor = torch.stack(test_image_list) if test_image_list else None

print(f"Train images tensor: {train_image_tensor.shape if train_image_tensor else 'None'}")
print(f"Test images tensor: {test_image_tensor.shape if test_image_tensor else 'None'}")

In [ ]:
def visualize_sample_images(image_dict, title="Sample Images"):
    fig, axes = plt.subplots(1, 5, figsize=(20, 4))

    sample_ids = list(image_dict.keys())[:5]

    for idx, (ax, img_id) in enumerate(zip(axes, sample_ids)):
        img_tensor = image_dict[img_id]
        img_np = img_tensor.permute(1, 2, 0).numpy()
        img_np = (img_np * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]
        img_np = np.clip(img_np, 0, 1)

        ax.imshow(img_np)
        ax.set_title(f"ID: {img_id[:10]}...")
        ax.axis('off')

    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

visualize_sample_images(train_images, "Sample Training Images")
visualize_sample_images(test_images, "Sample Test Images")

In [ ]:
torch.save({
    'train_tabular': train_tabular_tensor,
    'train_prices': train_price_tensor,
    'train_images': train_image_tensor,
    'test_tabular': test_tabular_tensor,
    'test_images': test_image_tensor,
    'scaler': tab_scaler,
    'encoder': zip_encoder,
    'train_tabular_cols': train_tabular.drop('price', axis=1).columns.tolist()
}, 'processed_data.pt')

print("Preprocessing complete! Data saved to processed_data.pt")